In [1]:
#This code will parse through the Peak_ID worksheet and generate another worksheet in the same file called "External_STD"
#with concentration and peak area data for C7, C8, ..., C17 arranged like so:
#
# Chain Length | Conc (mg/L) | Peak Area | Conc (mg/L) | Peak Area | Conc (mg/L) | Peak Area | Conc (mg/L) | Peak Area
# _____________________________________________________________________________________________________________________
#      C7      |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#      C8      |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#      ...     |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#      C17     |    1000     |     #     |     500     |     #     |     100     |     #     |     25      |     #
#
#Peak_ID worksheet lay out...
#
#

In [ ]:
def getExtStdData(file, sheetname, area_col, chain_col):
    col = [area_col, chain_col]
    wb = openpyxl.load_workbook(file)
    sheet = wb[sheetname]
    #create dataframe from relevant columns only
    df = pd.read_excel(file, sheetname=sheetname, usecols = col)
    #convert dataframe to allow array traversals
    dfList = df.values.tolist()
    
    #create master matrix holding arrays of area values
    num_conc, num_chain = 5, 11;
    Master = [[0 for x in range(num_conc)] for y in range(num_chain)] 
    
    #loop through area column, creating different arrays at the start of a non-numerical value (indicates new conc).
    #  Populate master matrix at end of each loop. In the same loop, populate conc matrix with all conc values using 
    #  non-numerical value title (new title indicates new conc).
    #  Conc matrix should be the same size as area matrix. At the same time create dictionary?
    
    #parse through dictionary to create data table seen in first cell
    #To create data table...
    